<a href="https://colab.research.google.com/github/ychoi-kr/gpt5-chatgpt/blob/main/8_finetuning/5_distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [4]:
from openai import OpenAI

client = OpenAI()

In [1]:
# 패키지 설치
!pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
from datasets import load_dataset

# 데이터셋 준비
dataset = load_dataset("ychoikr/Ani-Bench-JP-ko", split="test")
print(dataset[0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

{'문제': '주인공의 이름은 무엇입니까?', '답변': '카나메 마도카', '프로그램명': '마법소녀 마도카☆마기카'}


In [5]:
import json

# 학습 데이터 생성
with open("train.jsonl", "w", encoding="utf-8") as f:
    for entry in dataset:
        question_text = f"「{entry['프로그램명']}」에 관한 문제입니다. {entry['문제']}"

        # 추론 실행
        response = client.responses.create(
            model="gpt-4.1",
            input=[{"role": "user", "content": question_text}],
        )
        new_entry = {"messages": [
            {"role": "user", "content": question_text},
            {"role": "assistant", "content": response.output_text.replace("\n", "\\n")}
        ]}
        print(new_entry)
        f.write(json.dumps(new_entry, ensure_ascii=False) + "\n")


{'messages': [{'role': 'user', 'content': '「마법소녀 마도카☆마기카」에 관한 문제입니다. 주인공의 이름은 무엇입니까?'}, {'role': 'assistant', 'content': '「마법소녀 마도카☆마기카」의 주인공 이름은 **카나메 마도카(鹿目まどか, Kaname Madoka)**입니다.'}]}
{'messages': [{'role': 'user', 'content': '「마법소녀 마도카☆마기카」에 관한 문제입니다. 마도카의 친구이자 파란 머리의 마법소녀는 누구입니까?'}, {'role': 'assistant', 'content': '「마법소녀 마도카☆마기카」에서 마도카의 친구이자 파란 머리의 마법소녀는 **미키 사야카(미키 사야카, 美樹 さやか)**입니다.'}]}
{'messages': [{'role': 'user', 'content': '「마법소녀 마도카☆마기카」에 관한 문제입니다. 큐베의 본명은 무엇입니까?'}, {'role': 'assistant', 'content': '「마법소녀 마도카☆마기카」에 등장하는 큐베(QB, キュゥべえ)의 본명은 **"인큐베이터(Incubator, インキュベーター)"** 입니다.\\n\\n劇중에서 밝혀지는 큐베의 정체이자 종족명(본명)이 바로 인큐베이터입니다.'}]}
{'messages': [{'role': 'user', 'content': '「마법소녀 마도카☆마기카」에 관한 문제입니다. 마도카가 처음으로 마법소녀 계약을 제안받은 것은 몇 화입니까?'}, {'role': 'assistant', 'content': '「마법소녀 마도카☆마기카」에서 마도카가 큐베(QB)로부터 처음으로 마법소녀 계약을 제안받는 장면은 **제1화**에서 나옵니다.\\n\\n구체적으로, 제1화 「꿈을 구운 머핀 (꿈을 보는 머핀)」에서 마도카와 사야카는 학교 복도에서 큐베와 처음 만나고, 마법소녀 계약에 대해 큐베로부터 설명을 듣게 됩니다. 즉, 마도카가 마법소녀가 될 것을 처음으로 권유받는 것은 1화입니다.'}